In [58]:
import pandas as pd
import numpy as np

In [59]:
import math
import torch
import torch.nn.functional as F
from torch_geometric.nn import ChebConv
from torch_geometric.utils import to_dense_adj, dense_to_sparse
from torch_geometric.nn.conv import MessagePassing

In [60]:
lines = pd.read_csv(r"C:\Users\Dell\Documents\GitHub\pinns_opf\data\raw\lines_33.csv")
nodes = pd.read_csv(r"C:\Users\Dell\Documents\GitHub\pinns_opf\data\raw\nodes_33.csv")

In [61]:
node_index = nodes['Nodes'].values

In [62]:
file_path = "data.npy"

with open(file_path, "rb") as file:
    data_array = np.load(file)

In [63]:
# Assuming you already have the 'data_array' with shape (24, num_columns)
num_copies = 100

# Create an array of shape (num_copies, 1) to repeat 'data_array' vertically
repeated_array = np.tile(data_array, (num_copies, 1))

# Stack the repeated arrays vertically
data_array_extended = np.vstack(repeated_array)

print(data_array_extended.shape)

(2400, 203)


In [64]:
features = data_array_extended[1:]
targets = data_array_extended[:-1]
features.shape,targets.shape

((2399, 203), (2399, 203))

In [65]:
edge_index = np.array([lines['From'].values,lines['To'].values])
edge_index.shape

(2, 32)

In [66]:
resistance =  np.array(lines['R'].values)
reactance = np.array(lines['X'].values)
static_edge_features = np.stack([resistance, reactance]).transpose()
static_edge_features.shape

(32, 2)

In [67]:
edge_weights = np.ones((32,1))

In [68]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [69]:
dataset = StaticGraphTemporalSignal(
    edge_index = edge_index, edge_weight = edge_weights ,features = features, targets = targets
)

In [70]:
import numpy as np
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split

# GPU support
DEVICE = torch.device('cpu') # cuda
shuffle=True
batch_size = 32

In [88]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [72]:
class GConvGRU(torch.nn.Module):
    
#------------------------------------------------------------------------init
    def __init__( self, in_channels: int, out_channels: int, K: int,normalization: str = "sym",bias: bool = True ):
        super(GConvGRU, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.normalization = normalization
        self.bias = bias
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):
        self.conv_x_z = ChebConv(in_channels=self.in_channels,out_channels=self.out_channels, K=self.K, normalization=self.normalization, bias=self.bias)
        self.conv_h_z = ChebConv(in_channels=self.out_channels,out_channels=self.out_channels, K=self.K, normalization=self.normalization, bias=self.bias)

    def _create_reset_gate_parameters_and_layers(self):
        self.conv_x_r = ChebConv( in_channels=self.in_channels,out_channels=self.out_channels, K=self.K,  normalization=self.normalization, bias=self.bias )
        self.conv_h_r = ChebConv( in_channels=self.out_channels,out_channels=self.out_channels, K=self.K,  normalization=self.normalization, bias=self.bias )

    def _create_candidate_state_parameters_and_layers(self):
        self.conv_x_h = ChebConv(  in_channels=self.in_channels, out_channels=self.out_channels, K=self.K,normalization=self.normalization,bias=self.bias)
        self.conv_h_h = ChebConv(  in_channels=self.out_channels, out_channels=self.out_channels, K=self.K,normalization=self.normalization,bias=self.bias)

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()
        
#-------------------------------------------------------------------------------------------
    def _set_hidden_state(self, X): # step 1
        H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H
#---------------------------------------------------
    def _calculate_update_gate(self, X, edge_index, edge_weight, H): # step 2
        Z = self.conv_x_z(X, edge_index, edge_weight)
        Z = Z + self.conv_h_z(H, edge_index, edge_weight)
        Z = torch.sigmoid(Z)
        return Z
#------------------------------------------------------
    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = self.conv_x_r(X, edge_index, edge_weight)
        R = R + self.conv_h_r(H, edge_index, edge_weight)
        R = torch.sigmoid(R)
        return R
#------------------------- # Step 4
    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = self.conv_x_h(X, edge_index, edge_weight)
        H_tilde = H_tilde + self.conv_h_h(H * R, edge_index, edge_weight)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

#-------------------------------
    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward( self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X) # step 1 # X (20,4) H (20,32)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H) # step 2 Z (20, 32)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H) # step 3  R (20, 32)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R) # step 4 H_tilde (20, 32)
        
        H = self._calculate_hidden_state(Z, H, H_tilde) # step 5  H (20, 32)
        return H

In [73]:
class DConv(MessagePassing):
    
#----------------------------------------- init
    def __init__(self, in_channels, out_channels, K, bias=True):
        super(DConv, self).__init__(aggr="add", flow="source_to_target")
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = torch.nn.Parameter(torch.Tensor(2, K, in_channels, out_channels))  # 2, 32, 20,1 ?

        if bias:
            self.bias = torch.nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter("bias", None)

        self.__reset_parameters()

    def __reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.zeros_(self.bias)

#---------------------------------------------------
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor) -> torch.FloatTensor:
        
        adj_mat = to_dense_adj(edge_index, edge_attr=edge_weight) # create  adjacency matrix shape (1,20,20)
        adj_mat = adj_mat.reshape(adj_mat.size(1), adj_mat.size(2)) # (20, 20)
        deg_out = torch.matmul(
            adj_mat, torch.ones(size=(adj_mat.size(0), 1)).to(X.device)
        ) #  (20, 1)
        deg_out = deg_out.flatten() # (20)
        deg_in = torch.matmul(
            torch.ones(size=(1, adj_mat.size(0))).to(X.device), adj_mat
        ) # (1, 20)
        deg_in = deg_in.flatten() # 20 

        deg_out_inv = torch.reciprocal(deg_out) # receprocal of each item 2 will be 1/2 (20)
        deg_in_inv = torch.reciprocal(deg_in) # (20)
        row, col = edge_index
        
        norm_out = deg_out_inv[row] # (102)
        norm_in = deg_in_inv[row] # (102)

        reverse_edge_index = adj_mat.transpose(0, 1) # (20,20)
        reverse_edge_index, vv = dense_to_sparse(reverse_edge_index) #  creates sparse adjacency matrix defined by edge indices and edge attributes. # (2, 102), (102)

        Tx_0 = X  # (20, 36)
        Tx_1 = X  # (20, 36)
        # weight is (2, 2, 36, 32) so weight[0][0] is (36, 32) 
        H = torch.matmul(Tx_0, (self.weight[0])[0]) +  torch.matmul(Tx_0, (self.weight[1])[0]) # (20, 32) # calculate the embedding of each node -> step1

        # Step 2 message passing
        if self.weight.size(1) > 1:
            Tx_1_o = self.propagate(edge_index, x=X, norm=norm_out, size=None) # (20, 36)
            Tx_1_i = self.propagate(reverse_edge_index, x=X, norm=norm_in, size=None) # (20, 36)
            H = ( H  + torch.matmul(Tx_1_o, (self.weight[0])[1]) + torch.matmul(Tx_1_i, (self.weight[1])[1]) ) #(20, 32)
            
        #for k in range(2, self.weight.size(1)): # not true
        #    Tx_2_o = self.propagate(edge_index, x=Tx_1_o, norm=norm_out, size=None)
        #    Tx_2_o = 2.0 * Tx_2_o - Tx_0  # (20, 36)
        #    Tx_2_i = self.propagate(reverse_edge_index, x=Tx_1_i, norm=norm_in, size=None)
        #    Tx_2_i = 2.0 * Tx_2_i - Tx_0
        #    H = (H + torch.matmul(Tx_2_o, (self.weight[0])[k]) + torch.matmul(Tx_2_i, (self.weight[1])[k]))
        #    Tx_0, Tx_1_o, Tx_1_i = Tx_1, Tx_2_o, Tx_2_i

        if self.bias is not None:
            H += self.bias

        return H

In [74]:
class DCRNN(torch.nn.Module):
    #----------------------------------------------------init
    def __init__(self, in_channels: int, out_channels: int, K: int, bias: bool = True):
        super(DCRNN, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.bias = bias
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):
        self.conv_x_z = DConv(in_channels=self.in_channels + self.out_channels, out_channels=self.out_channels, K=self.K, bias=self.bias)

    def _create_reset_gate_parameters_and_layers(self):
        self.conv_x_r = DConv( in_channels=self.in_channels + self.out_channels,out_channels=self.out_channels, K=self.K, bias=self.bias)

    def _create_candidate_state_parameters_and_layers(self):
        self.conv_x_h = DConv( in_channels=self.in_channels + self.out_channels, out_channels=self.out_channels, K=self.K, bias=self.bias)

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()
#--------------------------------------------------------------------------

    def _set_hidden_state(self, X, H): # step 1
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H): # step 2
        Z = torch.cat([X, H], dim=1) # (20, 36)
        Z = self.conv_x_z(Z, edge_index, edge_weight) # (20, 32)
        Z = torch.sigmoid(Z)

        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H): # step 3
        R = torch.cat([X, H], dim=1)
        R = self.conv_x_r(R, edge_index, edge_weight)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R): # step 4
        H_tilde = torch.cat([X, H * R], dim=1)
        H_tilde = self.conv_x_h(H_tilde, edge_index, edge_weight)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde): # step 5
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward( self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None, H: torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X, H)  # X(20,4) H(20, 32)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H) # Z (20,32)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H) # R (20,32)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R) # H_tilde (20, 32)
        H = self._calculate_hidden_state(Z, H, H_tilde) # H (20, 32)
        return H

In [75]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, filters, 2) # or use  GConvGRU(node_features, filters, 2) both will lead to similar results
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight) # h (20, 32)
        h = F.relu(h) # h (20, 32)

        h = self.linear(h) # h (20, 1)
        return h

In [91]:
from tqdm import tqdm

model = RecurrentGCN(node_features=4, filters=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [87]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
loader = ChickenpoxDatasetLoader()
dataset = loader.get_dataset()
dataset.x.shape, dataset.edge_index.shape , dataset.edge_attr.shape

AttributeError: 'StaticGraphTemporalSignal' object has no attribute 'x'

In [90]:
snapshot.x.shape, snapshot.edge_index.shape , snapshot.edge_attr.shape

(torch.Size([20, 4]), torch.Size([2, 102]), torch.Size([102]))

In [92]:
model.train()

for epoch in range(100):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print("epoch {}--cost is {}".format(epoch, round(cost.item(),6)))

epoch 0--cost is 1.356619
epoch 1--cost is 1.331168
epoch 2--cost is 1.342036
epoch 3--cost is 1.312562
epoch 4--cost is 1.313701
epoch 5--cost is 1.306513
epoch 6--cost is 1.306989
